# Radar Data - the Current State of the (Py)-ART

In [88]:
import open_radar_data
import pyart
import dask.array as da
import hvplot.xarray
from distributed import Client
import dask.bag as db
import gc

In [71]:
client = Client()
client

2023-03-13 10:18:25,477 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/bw/c9j8z20x45s2y20vv6528qjc0000gq/T/dask-worker-space/worker-1gvix497', purging
2023-03-13 10:18:25,478 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/bw/c9j8z20x45s2y20vv6528qjc0000gq/T/dask-worker-space/worker-mup7nkgm', purging
2023-03-13 10:18:25,479 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/bw/c9j8z20x45s2y20vv6528qjc0000gq/T/dask-worker-space/worker-g3okswag', purging
2023-03-13 10:18:25,479 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/bw/c9j8z20x45s2y20vv6528qjc0000gq/T/dask-worker-space/worker-pz7sjpv_', purging
2023-03-13 10:18:25,480 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/bw/c9j8z20x45s2y20vv6528qjc0000gq/T/dask-worker-space/worker-8n3cu95e', purging
2023-03-13 10:18:25,480 - distributed.diskutils - INFO - Found st

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 10,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54219,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:54245,Total threads: 2
Dashboard: http://127.0.0.1:54246/status,Memory: 6.40 GiB
Nanny: tcp://127.0.0.1:54222,


In [2]:
from open_radar_data import DATASETS

In [3]:
files = ['gucxprecipradarcmacppiS2.c1.20220314.021559.nc',
         'gucxprecipradarcmacppiS2.c1.20220314.024239.nc',
         'gucxprecipradarcmacppiS2.c1.20220314.025840.nc']

In [4]:
filepaths = [DATASETS.fetch(file) for file in files]

In [91]:
%%time
radar = pyart.io.read(filepaths[0])

/Users/mgrover/miniforge3/envs/radar-dask-demo-dev/lib/python3.11/site-packages/pyart/io/cfradial.py:408: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  data = self.ncvar[:]
/Users/mgrover/miniforge3/envs/radar-dask-demo-dev/lib/python3.11/site-packages/pyart/io/cfradial.py:408: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  data = self.ncvar[:]


CPU times: user 860 ms, sys: 91 ms, total: 951 ms
Wall time: 919 ms


In [58]:
%%time
grid = pyart.map.grid_from_radars(
    (radar,),
    grid_shape=(18, 160, 160),
    grid_limits=((500, 5000), (-20_000.,20_000.),(-20_000.,20_000.),),
    fields=["corrected_reflectivity"],
)

CPU times: user 241 ms, sys: 14.5 ms, total: 256 ms
Wall time: 261 ms


In [49]:
radar.fields['corrected_reflectivity']['data'] = da.asarray(radar.fields['corrected_reflectivity']['data'])

In [53]:
%%time
grid = pyart.map.grid_from_radars(
    (radar,),
    grid_shape=(18, 160, 160),
    grid_limits=((500, 5000), (-20_000.,20_000.),(-20_000.,20_000.),),
    fields=["corrected_reflectivity"],
)

CPU times: user 218 ms, sys: 8.92 ms, total: 226 ms
Wall time: 232 ms


In [59]:
ds = grid.to_xarray()

In [86]:
def compute_number_of_points(extent, resolution):
    """
    Create a helper function to determine number of points
    """
    return int((extent[1] - extent[0])/resolution) + 1

def grid_radar(file,
               x_grid_limits=(-20_000.,20_000.),
               y_grid_limits=(-20_000.,20_000.),
               z_grid_limits = (500.,5_000.),
               grid_resolution = 250,

               ):
    """
    Grid the radar using some provided parameters
    """

    try:
        radar = pyart.io.read(file)
    except KeyError:
        print('Issue with reading latitude for ', file)


    x_grid_points = compute_number_of_points(x_grid_limits, grid_resolution)
    y_grid_points = compute_number_of_points(y_grid_limits, grid_resolution)
    z_grid_points = compute_number_of_points(z_grid_limits, grid_resolution)

    grid = pyart.map.grid_from_radars(radar,
                                      grid_shape=(z_grid_points,
                                                  y_grid_points,
                                                  x_grid_points),
                                      grid_limits=(z_grid_limits,
                                                   y_grid_limits,
                                                   x_grid_limits),
                                      method='nearest',
                                      constant_roi=250,
                                     )
    del radar
    return grid.to_xarray()

def run_squire(file):
    """
    Runs the SQUIRE workflow
    """

    # Read the file, and grid to a cartesian grid
    ds = grid_radar(file)

    # Create an output path
    out_path = f"data/{Path(file).stem}.gridded.nc"

    # Write the dataset to a netcdf file in the specified directory
    ds.to_netcdf(out_path)

    # Close and delete everything
    ds.close()

    del ds
    gc.collect()

In [87]:
run_squire(filepaths[0])

/Users/mgrover/miniforge3/envs/radar-dask-demo-dev/lib/python3.11/site-packages/pyart/io/cfradial.py:408: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  data = self.ncvar[:]
/Users/mgrover/miniforge3/envs/radar-dask-demo-dev/lib/python3.11/site-packages/pyart/io/cfradial.py:408: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  data = self.ncvar[:]


In [92]:
%%time
file_bag = db.from_sequence(filepaths, 1)
computation = file_bag.map(run_squire)
computation.compute()

/Users/mgrover/miniforge3/envs/radar-dask-demo-dev/lib/python3.11/site-packages/pyart/io/cfradial.py:408: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  data = self.ncvar[:]
/Users/mgrover/miniforge3/envs/radar-dask-demo-dev/lib/python3.11/site-packages/pyart/io/cfradial.py:408: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  data = self.ncvar[:]
/Users/mgrover/miniforge3/envs/radar-dask-demo-dev/lib/python3.11/site-packages/pyart/io/cfradial.py:408: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  data = self.ncvar[:]
/Users/mgrover/miniforge3/envs/radar-dask-demo-dev/lib/python3.11/site-packages/pyart/io/cfradial.py:408: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  data = self.ncvar[:]
/Users/mgrover/miniforge3/envs/radar-dask-demo-dev/lib/python3.11/site-packages/pyart/io/cfradial.py:408: Us

CPU times: user 178 ms, sys: 43.8 ms, total: 222 ms
Wall time: 3.06 s


[None, None, None]

In [69]:
ds.corrected_reflectivity.squeeze().hvplot.quadmesh(x='lon',
                                                    y='lat',
                                                    cmap='Spectral_r',
                                                    width=600,
                                                    height=500,
                                                    geo=True,
                                                    vmin=-20,
                                                    vmax=40)

/Users/mgrover/miniforge3/envs/radar-dask-demo-dev/lib/python3.11/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/mgrover/miniforge3/envs/radar-dask-demo-dev/lib/python3.11/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/mgrover/miniforge3/envs/radar-dask-demo-dev/lib/python3.11/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/mgrover/miniforge3/envs/radar-dask-demo-dev/lib/python3.11/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


/Users/mgrover/miniforge3/envs/radar-dask-demo-dev/lib/python3.11/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


:DynamicMap   [z]
   :QuadMesh   [lon,lat]   (corrected_reflectivity)